In [9]:
import psycopg2

In [41]:
conn = psycopg2.connect(database="nepse",
                        host="localhost",
                        user="postgres",
                        password="1234",
                        port="5433")

In [42]:
cursor = conn.cursor()

In [30]:
def create_tables():
    """ Create tables in the PostgreSQL database """
    commands = (
        """ CREATE TABLE IF NOT EXISTS COMPANIES (
                id SERIAL PRIMARY KEY,
                name TEXT UNIQUE,
                sector TEXT  -- e.g., Bank, Hydropower
            )
        """,
        """ CREATE TABLE IF NOT EXISTS REPORTS (
                id SERIAL PRIMARY KEY,
                company_id INT REFERENCES COMPANIES(id) ON DELETE CASCADE,
                report_type TEXT CHECK (report_type IN ('Balance Sheet', 'Profit & Loss', 'Key Metrics', 'Ratio Analysis', 'Others')),
                fiscal_year INT,
                quarter INT,
                uploaded_at TIMESTAMP DEFAULT NOW(),
                UNIQUE (company_id, fiscal_year, quarter, report_type)
            )
        """,
        """ CREATE TABLE IF NOT EXISTS REPORT_DATA (
                id SERIAL PRIMARY KEY,
                report_id INT REFERENCES REPORTS(id) ON DELETE CASCADE,
                metric TEXT,  -- Column name from CSV (e.g., 'Total Assets', 'Net Profit')
                value NUMERIC,  -- The actual numerical value of the metric
                unit TEXT  -- Currency (NPR), percentage (%), or ratio
            )
        """
    )
    try:
        # Execute each command in a transaction
        for command in commands:
            cursor.execute(command)

        # Commit changes
        conn.commit()
        print("Tables created successfully.")

    except (Exception, psycopg2.DatabaseError) as error:
        print(f"Error: {error}")
        if conn:
            conn.rollback()  # Rollback on error

    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()


In [31]:
create_tables()

Tables created successfully.


In [45]:
# Drop table COMPANIES
sql = '''DROP TABLE COMPANIES'''

cursor.execute(sql)
conn.commit()


In [46]:
cursor.close()
conn.close()

In [ ]:
# Create table companies
data = []